# <div align = "center" style="color:rgb(250, 250, 20);"> lat_OS_and_brand </div>

# Define Libraries

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')


from datetime import datetime

# %% [markdown]
## Configure Settings
# Set options or configurations as needed

# Example: pd.set_option('display.max_columns', None)



### Documentation: Understanding the OS Version Query Logic

This query combines device OS information from Android/iOS Credolab datasets with a loan master table, then selects the first record per loan account. Below is a step-by-step breakdown:

---

#### **1. CTE: `osversion_credolab`**
```sql
osversion_credolab as (
  SELECT * FROM (
    -- Android devices
    SELECT DISTINCT 
      deviceId,
      'android' || generalInfo.release AS credolab_osversion,  -- Prefixes OS version with "android"
      generalInfo.brand
    FROM credolab_raw.android_credolab_datasets_struct_columns
    
    UNION ALL
    
    -- iOS devices
    SELECT DISTINCT 
      deviceId,
      'ios' || generalInfo.release AS credolab_osversion,  -- Prefixes OS version with "ios"
      generalInfo.brand
    FROM credolab_raw.ios_credolab_datasets_struct_columns
  )
)
```
**Purpose**:  
- Combines **Android** and **iOS** device data into a unified dataset.
- **Structure**:
  - `deviceId`: Unique device identifier.
  - `credolab_osversion`: OS version prefixed with `android`/`ios` (e.g., `android12`, `ios15`).
  - `brand`: Device manufacturer (e.g., Samsung, Apple).
- Uses `UNION ALL` to retain duplicates (if any) for performance.
- `DISTINCT` ensures unique records per source table.

---

#### **2. CTE: `ocb1`**
```sql
ocb1 as (
  SELECT 
    lmt.digitalLoanAccountid,
    lmt.credolabRefNumber,
    lmt.credolabDeviceId,
    oc.deviceid,
    oc.credolab_osversion,
    oc.brand,
    ROW_NUMBER() OVER (
      PARTITION BY digitalLoanAccountId 
      ORDER BY digitalLoanAccountId
    ) AS rnk
  FROM `risk_credit_mis.loan_master_table` lmt 
  LEFT JOIN osversion_credolab oc 
    ON oc.deviceId = lmt.credolabRefNumber  -- Joins via Credolab reference ID
)
```
**Purpose**:  
- Links loan accounts to device OS data.
- **Key Operations**:
  - **Left Join**: Includes all loans from `loan_master_table`, even if no matching device exists.
  - **Row Numbering**: Assigns a rank (`rnk`) to each row partitioned by `digitalLoanAccountId`:
    - `PARTITION BY digitalLoanAccountId`: Groups records by loan account.
    - `ORDER BY digitalLoanAccountId`: Since ordering uses the partition key, the order is **arbitrary** within groups.
  - **Fields**:
    - Loan identifiers: `digitalLoanAccountid`, `credolabRefNumber`, `credolabDeviceId`.
    - Device info: `deviceid`, `credolab_osversion`, `brand`.

---

#### **3. Final Result**
```sql
SELECT * 
FROM ocb1 
WHERE rnk = 1
```
**Purpose**:  
- Filters results to **one record per loan account** (`digitalLoanAccountId`).
- `rnk = 1` selects the first row in each partition (arbitrarily due to ordering logic).

---

### **Key Business Logic**
1. **Unified Device Data**:  
   Combines Android/iOS datasets to create a single source for device OS versions and brands.

2. **Loan-Device Linkage**:  
   Uses `credolabRefNumber` (loan reference ID) to match devices from Credolab data.

3. **Deduplication Strategy**:  
   - `rnk = 1` ensures **one record per loan account**, even if multiple devices match.
   - ⚠️ **Ordering Limitation**: Rows are ordered arbitrarily within partitions. Add explicit ordering (e.g., timestamp) if prioritization is needed.

---

### **Potential Improvements**
1. **Explicit Ordering**:  
   Modify the `ROW_NUMBER()` to use a meaningful sort order (e.g., event time) instead of `digitalLoanAccountId`:
   ```sql
   ROW_NUMBER() OVER (
     PARTITION BY digitalLoanAccountId 
     ORDER BY event_timestamp DESC  -- Example: Prefer latest device
   ) AS rnk
   ```

2. **Handle NULLs**:  
   Add logic for cases where `credolabRefNumber` has no matching device (e.g., `COALESCE(oc.brand, 'Unknown')`).

3. **Optimize Deduplication**:  
   Use `GROUP BY` in `osversion_credolab` if duplicate `deviceId` entries exist across platforms.

---

### **Output Columns**
| Column                 | Source Table         | Description                          |
|------------------------|----------------------|--------------------------------------|
| `digitalLoanAccountid` | `loan_master_table`  | Unique loan account ID.              |
| `credolabRefNumber`    | `loan_master_table`  | Device reference ID (join key).      |
| `credolabDeviceId`     | `loan_master_table`  | Alternate device ID (unused in join).|
| `deviceid`             | `osversion_credolab` | Matched device ID.                   |
| `credolab_osversion`   | `osversion_credolab` | OS version (e.g., `android12`).      |
| `brand`                | `osversion_credolab` | Device brand (e.g., `Samsung`).      |

This query provides a foundational view linking loans to device OS data, with one record per loan account.

# Query

In [2]:
query = """
create or replace table `dap_ds_poweruser_playground.lat_osandbrand` as 
with 
osversion_credolab as (select * from (
(select distinct deviceId,'android'||generalInfo.release as credolab_osversion,  generalInfo.brand from credolab_raw.android_credolab_datasets_struct_columns
union all
select distinct deviceId,'ios'||generalInfo.release as credolab_osversion,  generalInfo.brand from  credolab_raw.ios_credolab_datasets_struct_columns))),
ocb1 as 
(select 
lmt.digitalLoanAccountid,
lmt.credolabRefNumber,
lmt.credolabDeviceId,
oc.deviceid,
COALESCE(oc.credolab_osversion, 'Unknown') AS credolab_osversion,
COALESCE(oc.brand, 'Unknown')brand,
row_number() over(partition by digitalLoanAccountId order by digitalLoanAccountId) rnk
from `risk_credit_mis.loan_master_table` lmt 
left join osversion_credolab oc on oc.deviceId = lmt.credolabRefNumber
)
select * from ocb1 where rnk = 1"""


query_job = client.query(query)
# Wait for the job to complete.
query_job.result()
print(f"Created table: {query_job.destination}")

Created table: prj-prod-dataplatform.dap_ds_poweruser_playground.lat_osandbrand
